In [2]:
class bearing4:
    da = {}
    de,fe,ba = [],[],[]
    sampling_rate = 0
    
    def __init__(self,ad,rate):    
        self.da = io.loadmat(str(ad)+'.mat')
        self.de = np.array([x[0] for x in self.da['X'+str(ad)+'_DE_time']])
        self.fe = np.array([x[0] for x in self.da['X'+str(ad)+'_FE_time']])
        self.ba = np.array([x[0] for x in self.da['X'+str(ad)+'_BA_time']])
        self.sampling_rate = rate
        return
    
    def STFT_img1(self,data,name = 'hann'):
        #短时傅里叶变换
        f, t, zxx = signal.stft(data,fs = self.sampling_rate,window = 'hann',nperseg = 128,noverlap=127)
        plt.figure(figsize = (12, 4))
        plt.contourf(t,f, np.abs(zxx))
        plt.colorbar()
        plt.title('STFT Manitude' + '-' + name)
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        fig = plt.figure(figsize=(12, 9))
        axes3d = Axes3D(fig)
        print(t.shape,f.shape,zxx.shape)
        T,F=np.meshgrid(t,f)
        print(F.shape,T.shape,zxx.shape)
        axes3d.plot_surface(F,T, np.abs(zxx))
        axes3d.set_xlabel('Frequency [Hz]')
        axes3d.set_ylabel('Time [sec]')
        axes3d.set_zlabel('STFT')
        return 
    
    def CWT_img2(self,data,name = 'morl',wavename = 'morl'):
        #连续小波分析
        '''墨西哥帽小波"mexh"
        Morlet小波"morl"
        复数Morlet小波（"cmorB-C"浮点值B，C）
        高斯小波（"gausP"其中P是1到8之间的整数）
        复数高斯小波（"cgauP"其中P是1到8之间的整数）
        香农小波（"shanB-C"具有浮点值B和C）哪里 B 是带宽和 C 是中心频率。
        频率B样条小波（"fpspM-B-C"具有整数M和浮点B，C）
        M  是样条顺序， B 是带宽和 C 是中心频率。
        '''
        T = 1.0 / self.sampling_rate
        #采样周期
        t = np.arange(0, len(data) * T, T)
        fc = pywt.central_frequency(wavename)
        #中心频率
        totalscal = 20
        #小波尺度
        cparam = 2 * fc * totalscal
        #计算小波尺度
        scales = cparam / np.arange(totalscal, 0, -1)
        [cwtmatr, frequencies] = pywt.cwt(data, scales, wavename, T)
        plt.figure(figsize = (12, 4))
        plt.plot(t, data)
        plt.figure(figsize = (12, 4))
        plt.contourf(t, frequencies, abs(cwtmatr))
        plt.colorbar()
        plt.title('CWT Manitude' + '-' + name)
        plt.ylabel('Frequency [Hz]')
        plt.xlabel('Time [sec]')
        return
    
    def CYWT_img3(self,data,wavelet,maxlevel = 3):
        #小波包分析
        plt.rcParams['font.sans-serif'] = ['Simhei']
        #正常显示中文字体
        plt.rcParams['axes.unicode_minus'] = False
        #显示正负号
        energy = []
        wp = pywt.WaveletPacket(data = data,wavelet = wavelet,mode = 'symmetric',maxlevel = maxlevel)
        freqTree = [node.path for node in wp.get_level(maxlevel,'freq')]
        freqBand = self.sampling_rate / (2 ** maxlevel)

        fig,axes = plt.subplots(len(freqTree) + 1,1,figsize = (10,7))
        axes[0].plot(data)
        axes[0].set_title('原始数据')

        for i in range(len(freqTree)):
            axes[i + 1].plot(wp[freqTree[i]].reconstruct(update = True))
            axes[i + 1].set_title('S'+str(i + 1),y = 0.15,x = 1.02)
            energy.append(pow(np.linalg.norm(wp[freqTree[i]].data,ord=None),2))
        plt.figure(figsize = (12,4))
        plt.bar(np.arange(2 ** maxlevel),energy)
        plt.xlabel('频带段')
        plt.ylabel('振动能量')
        return
    
    def EMD_img4(self,data):
        #EMD分析
        T = 1.0 / self.sampling_rate  # 采样周期
        t = np.arange(0, len(data) * T, T)
        emd = EMD()
        emd.emd(data, t)
        # 提取IMF，与残差
        IMFs,res=emd.get_imfs_and_residue()
        # 绘制IMF
        vis1 = Visualisation()
        vis1.plot_imfs(imfs=IMFs,residue=res,t=t,include_residue=True)
        # 绘制并显示所提供IMF的瞬时频率
        vis2 = Visualisation()
        vis2.plot_instant_freq(t,imfs=IMFs)
print(6)

6


In [3]:
class bearing3:
    da = {}
    de,fe,ba = [],[],[]
    sampling_rate = 0
    
    def __init__(self,ad,rate):    
        self.da = io.loadmat(str(ad)+'.mat')
        self.de = np.array([x[0] for x in self.da['X'+str(ad)+'_DE_time']])
        self.fe = np.array([x[0] for x in self.da['X'+str(ad)+'_FE_time']])
        self.ba = np.array([x[0] for x in self.da['X'+str(ad)+'_BA_time']])
        self.sampling_rate = rate
        return
    
    def __pl132(self,x,y,xlabel,ylabel,title):
        #内部绘图函数
        plt.figure(figsize = (12,4))
        plt.plot(x,y)
        plt.xlabel(xlabel)
        plt.ylabel(ylabel)
        plt.title(title)
        return
    
    def __FFT(self,da):
        #内部傅里叶变换函数
        N = len(da)
        df = self.sampling_rate/N
        f = np.arange(0,self.sampling_rate / 2,df)
        y = fft(da)
        yf = abs(y)[0:len(f)]
        return [f,yf]
    
    def ft_img(self,da):
        #绘制信号的时域图像
        N = len(da)
        dt = 1 / self.sampling_rate
        t = np.arange(0,N*dt,dt)
        self.__pl(t,da,'t/s','x(t)','Signal')
        return
    
    def fft_img3(self,da,title):
        #对信号进行傅里叶变换并绘制其频谱图像
        f = self.__FFT(da)       
        self.__pl(f[0],f[1],'fs/Hz','al',str(title)  + ' FFT')
        return
    
    def corx_img(self,da):
        #对信号进行自相关分析并绘制其自相关图像和自谱图像
        cor_x = np.correlate(da,da,'full')
        de1 = np.arange(len(cor_x) / 2, 0, -1)
        de = np.hstack([de1[::-1], de1[1:]])
        dt=1/self.sampling_rate
        x = np.arange(-(len(cor_x)-1)/2*dt,((len(cor_x)-1)/2+1)*dt,dt)
        self.__pl(x,cor_x/de,'t/s','al','Autucorrelation')
        f = self.__FFT(cor_x/de)
        self.__pl(f[0],f[1],'fs/Hz','al','Self power spectrum')
        return
    
    def corxy_img1(self,dax,day):
        #对信号进行互相关分析并绘制其互相关图像和互谱图像
        cor_xy = np.correlate(dax,day,'full')
        N = len(cor_xy)
        x = range(-N//2,N//2)
        self.__pl(x,cor_xy,'t/s','al','Cross-correlation')
        f = self.__FFT(cor_xy)
        self.__pl(f[0],f[1],'fs/Hz','al','Cross power spectrum')
        return
    
    def pro_den_img2(self,da):
        #绘制信号的概率密度函数图像
        plt.figure(figsize = (12,4))
        sns.distplot(da,bins = 500)
        sns.kdeplot(da)
        plt.title('Probability density function')
        return
    
    def compare_fft3(self,da1,da2):
        #比较两个信号的频谱
        f1 = self.__FFT(da1)
        f2 = self.__FFT(da2)
        delta_f = [f2[1][i]-f1[1][i] for i in range(0,len(f1[0]))]
        self.__pl(f1[0],delta_f,'fs/Hz','delta_al','Compare FFT')
        return
print(5)

5


In [4]:
str = '0123456789'
print(str)
print(str[0:-1])
print(str[0])                 #20
print(str[2:5])
print(str[2:])
print(str[1:5:2])
print(str*2)
print(str+'abc')
print('hello\npython')
print(r'hello\npython')

0123456789
012345678
0
234
23456789
13
01234567890123456789
0123456789abc
hello
python
hello\npython


In [5]:
x = [1,2,3,4,5,6,7,8,9]
a = b = c = x[5]
print(a,b,c)
x[5] = 10
print(a,b,c)
a = 10
print(a,b,c)
a,b,c = 1,2,'123'
print(a,b,c)       

6 6 6
6 6 6
10 6 6
1 2 123


In [6]:
def reverseWords(input):
    inputWords = input.split(' ')
    inputWords = inputWords[-1::-1]            #70
    output = ' '.join(inputWords)
    return output
if __name__ == '__main__':
    input = 'I like python'
    rw = reverseWords(input)
    print(rw)
print(8)

python like I
8


In [7]:
tuple = ('abcd',123,1.23,'python',3.14)
tinytuple = (123,'runoob')
print(tuple)
print(tuple[0])                              #80
print(tuple[1:3])
print(tuple[2:])
print(tinytuple*2)
print(tuple + tinytuple)

('abcd', 123, 1.23, 'python', 3.14)
abcd
(123, 1.23)
(1.23, 'python', 3.14)
(123, 'runoob', 123, 'runoob')
('abcd', 123, 1.23, 'python', 3.14, 123, 'runoob')


In [8]:
a = set('abcdefgxyz')
b = set('cdefgababcrst')
print(a)
print(a - b)
print(a | b)
print(a & b)
print(a ^ b)

{'e', 'c', 'd', 'f', 'a', 'g', 'y', 'z', 'x', 'b'}
{'x', 'z', 'y'}
{'r', 'd', 'g', 'z', 'x', 'b', 't', 'e', 'c', 'f', 'a', 'y', 's'}
{'e', 'c', 'd', 'f', 'a', 'g', 'b'}
{'r', 't', 'y', 'z', 's', 'x'}


In [10]:
a = 21
b = 10
c = 0
c = a + b
print('1 - c 的值为：',c)
c = a - b                                     
print('2 - c 的值为：',c)
c = a * b
print('3 - c 的值为：',c)
c = a / b
print('4 - c 的值为：',c)
c = a // b
print('5 - c 的值为：',c)
c = a % b
print('6 - c 的值为：',c)
a = 2                                         
b = 3
c = a ** b
print('7 - c 的值为：',c)

1 - c 的值为： 31
2 - c 的值为： 11
3 - c 的值为： 210
4 - c 的值为： 2.1
5 - c 的值为： 2
6 - c 的值为： 1
7 - c 的值为： 8
